In [1]:
#imports
import pandas as pd
import torch
from collections import Counter
from transformers import AutoModel, AutoTokenizer
import unicodedata
from datasets import load_dataset
import itertools

import os
import re
import numpy as np 
from sklearn.metrics import accuracy_score

import transformers
from transformers import BertTokenizer, TFBertForMaskedLM

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

device

'cpu'

In [ ]:
!pip uninstall TensorFLow

# The Data

Converting test, train and val datasets which include kanji to be purely kana (furigana)

In [4]:
#data_files = {"train": "train_split.csv", "test": "test_split.csv", 'val': 'val_split.csv'}
dataset = load_dataset('universal_dependencies', 'ja_gsd')
train_split = dataset['train']
test_split = dataset['test']
val_split = dataset['validation']

train_split_df = pd.DataFrame(data=train_split, columns=train_split.features)
train_split_df = train_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
train_split_df

test_split_df = pd.DataFrame(data=test_split, columns=test_split.features)
test_split_df = test_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
test_split_df

val_split_df = pd.DataFrame(data=val_split, columns=val_split.features)
val_split_df = val_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
val_split_df

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7027 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/501 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/543 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to C:/Users/brand/.cache/huggingface/datasets/universal_dependencies/ja_gsd/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

,idx,text,tokens
0,dev-s1,ただし、50周年ソングに変更後は、EDも歌つきのものが使われた。,"[ただし, 、, 50, 周年, ソング, に, 変更, 後, は, 、, ED, も, 歌..."
1,dev-s2,私は初めてだったんだけど思っていたよりも魚は新鮮でした。,"[私, は, 初めて, だっ, た, ん, だ, けど, 思っ, て, い, た, より, ..."
2,dev-s4,セントラル・リーグ審判員の水落朋大は実兄。,"[セントラル, ・, リーグ, 審判, 員, の, 水落, 朋大, は, 実兄, 。]"
3,dev-s5,多彩なライブアクトとともに、祝日前の渋谷の夜を盛り上げてくれそうだ。,"[多彩, な, ライブ, アクト, と, とも, に, 、, 祝日, 前, の, 渋谷, の..."
4,dev-s6,今作品では大会でのパラメータUPが非常に大きく、ALL999への育成は回復アイテムを投与しつ...,"[今, 作品, で, は, 大会, で, の, パラメータ, UP, が, 非常, に, 大..."
...,...,...,...
496,dev-s507,お風呂大きくはないけれどお湯はきれい。,"[お, 風呂, 大きく, は, ない, けれど, お, 湯, は, きれい, 。]"
497,dev-s508,新生闇軍団頭領。,"[新生, 闇, 軍団, 頭領, 。]"
498,dev-s509,"当初の説明と実際の施設の状態が違うという点も,川島町の住民の証言とよく似ています。","[当初, の, 説明, と, 実際, の, 施設, の, 状態, が, 違う, と, いう,..."
499,dev-s510,それでいてこの対応である。,"[それ, で, い, て, この, 対応, で, ある, 。]"


In [17]:
val_split_df

,idx,text,tokens,kana
0,dev-s1,ただし、50周年ソングに変更後は、EDも歌つきのものが使われた。,"[ただし, 、, 50, 周年, ソング, に, 変更, 後, は, 、, ED, も, 歌...",ただし、50しゅうねんソングにへんこうごは、EDもかつきのものがしわれた。
1,dev-s2,私は初めてだったんだけど思っていたよりも魚は新鮮でした。,"[私, は, 初めて, だっ, た, ん, だ, けど, 思っ, て, い, た, より, ...",しはしょめてだったんだけどしっていたよりもぎょはしんせんでした。
2,dev-s4,セントラル・リーグ審判員の水落朋大は実兄。,"[セントラル, ・, リーグ, 審判, 員, の, 水落, 朋大, は, 実兄, 。]",セントラル・リーグしんはんいんのすいらくほうだいはじつけい。
3,dev-s5,多彩なライブアクトとともに、祝日前の渋谷の夜を盛り上げてくれそうだ。,"[多彩, な, ライブ, アクト, と, とも, に, 、, 祝日, 前, の, 渋谷, の...",たさいなライブアクトとともに、しゅくにちぜんのじゅうこくのやをせいりじょうげてくれそうだ。
4,dev-s6,今作品では大会でのパラメータUPが非常に大きく、ALL999への育成は回復アイテムを投与しつ...,"[今, 作品, で, は, 大会, で, の, パラメータ, UP, が, 非常, に, 大...",こんさくひんではだいかいでのパラメータUPがひじょうにだいきく、ALL999へのいくせいはか...
...,...,...,...,...
496,dev-s507,お風呂大きくはないけれどお湯はきれい。,"[お, 風呂, 大きく, は, ない, けれど, お, 湯, は, きれい, 。]",おふうろだいきくはないけれどおとうはきれい。
497,dev-s508,新生闇軍団頭領。,"[新生, 闇, 軍団, 頭領, 。]",しんせいあんぐんだんとうりょう。
498,dev-s509,"当初の説明と実際の施設の状態が違うという点も,川島町の住民の証言とよく似ています。","[当初, の, 説明, と, 実際, の, 施設, の, 状態, が, 違う, と, いう,...","とうしょのせつめいとじつさいのしせつのじょうたいがいうというてんも,せんとうちょうのじゅうみ..."
499,dev-s510,それでいてこの対応である。,"[それ, で, い, て, この, 対応, で, ある, 。]",それでいてこのたいおうである。


In [5]:
print(len(train_split))
print(len(test_split))
print(len(val_split))
#train_split['text']

7027
543
501


In [6]:
#load kanji data
kanji_df = pd.read_json("kanji.json")
kanji_df = kanji_df.drop(['strokes', 'grade', 'freq', 'jlpt_old', 'jlpt_new', 'meanings', 'wk_radicals', 'wk_readings_kun', 'wk_readings_on', 'wk_meanings', 'wk_level'], axis=0)

In [7]:
# As part of data collection, we need a dataset that is ONLY kana. 
# We could not find a dataset that had both kanji AND kana only version,
# so we took a dataset and converted the kanji to kana. 
# kanji to kana is less ambiguous with how on vs kun readings work, but there is still some abiguity.
# we recognize that here, in the dataset.
def unkanjify(df):
    new_sent = []
    for word in df['tokens']:
        #iterating list of chararacters
        for char in word:
            if char in kanji_df.columns:#is kanji
                if char in kanji_df.columns or char in kanji_df.columns:
                    if kanji_df[char].loc['readings_on']:
                        new_sent.append(kanji_df[char].loc['readings_on'][0])
                    else:
                        new_sent.append(char)
                #kun reading
                else:
                    for c in kanji_df[char].loc['readings_kun'][0]:
                        if c == '.':
                            continue
                        new_sent.append(c)
            else:#is not kanji
                new_sent.append(char)
    joined_sent = ''.join(new_sent)
    return joined_sent

# Applying the function to all splits (train, test and val)
train_split_df['kana'] = train_split_df.apply(lambda x: unkanjify(x), axis=1) 
test_split_df['kana'] = test_split_df.apply(lambda x: unkanjify(x), axis=1) 
val_split_df['kana'] = val_split_df.apply(lambda x: unkanjify(x), axis=1) 
train_split_df

,idx,text,tokens,kana
0,train-s1,ホッケーにはデンジャラスプレーの反則があるので、膝より上にボールを浮かすことは基本的に反則に...,"[ホッケー, に, は, デンジャラス, プレー, の, 反則, が, ある, の, で, ...",ホッケーにはデンジャラスプレーのはんそくがあるので、しつよりじょうにボールをふかすことはきほ...
1,train-s2,また行きたい、そんな気持ちにさせてくれるお店です。,"[また, 行き, たい, 、, そんな, 気持ち, に, さ, せ, て, くれる, お, ...",またこうきたい、そんなきじちにさせてくれるおてんです。
2,train-s3,手に持った特殊な刃物を使ったアクロバティックな体術や、揚羽と薄羽同様にクナイや忍具を使って攻...,"[手, に, 持っ, た, 特殊, な, 刃物, を, 使っ, た, アクロバティック, な...",しゅにじったとくしゅなじんぶつをしったアクロバティックなたいじゅつや、よううとはくうどうよう...
3,train-s4,"3年次にはトータルオフェンスで2,892ヤードを獲得し、これは大学記録となった。","[3, 年次, に, は, トータル, オフェンス, で, 2, ,, 892, ヤード, ...","3ねんじにはトータルオフェンスで2,892ヤードをかくとくし、これはだいがくきろくとなった。"
4,train-s5,葬儀の最中ですよ!,"[葬儀, の, 最中, です, よ, !]",そうぎのさいちゅうですよ!
...,...,...,...,...
7022,train-s7160,「さくらちゃん」と呼ばれている。,"[「, さくら, ちゃん, 」, と, 呼ば, れ, て, いる, 。]",「さくらちゃん」とこばれている。
7023,train-s7161,シェルマルケ氏は新憲法などをめぐり、アハメド暫定大統領と対立していた。,"[シェルマルケ, 氏, は, 新, 憲法, など, を, めぐり, 、, アハメド, 暫定,...",シェルマルケしはしんけんほうなどをめぐり、アハメドざんていだいとうりょうとたいりつしていた。
7024,train-s7162,"自らがオウム真理教ではない別のカルト団体に12年間所属していた経験をもとに,こう語りました。","[自ら, が, オウム, 真理, 教, で, は, ない, 別, の, カルト, 団体, に...",じらがオウムしんりきょうではないべつのカルトだんたいに12ねんかんしょぞくしていたけいけんを...
7025,train-s7163,紅い髪と同じ色の瞳という容姿に加え、眉と鎖骨の下の部分に二つずつ紅い球体が埋まっているという...,"[紅い, 髪, と, 同じ, 色, の, 瞳, と, いう, 容姿, に, 加え, 、, 眉...",こういはつとどうじしょくのどうというようしにかえ、びとさこつのかのぶぶんににつずつこういきゅ...


In [52]:
#tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
#model = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")

tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
model = TFBertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


ImportError: 
TFBertForMaskedLM requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


In [39]:
inputsKanji = tokenizer(train_split_df['text'].values.tolist(), max_length=100, truncation=True, padding='max_length', return_tensors='np')
inputsKana = tokenizer(train_split_df['kana'].values.tolist(), max_length=100, truncation=True, padding='max_length', return_tensors='np')

In [40]:
inputsKana['labels'] = inputsKanji['input_ids']

In [47]:
model.to(device)    

optimizer_bert = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

NameError: name 'LEARNING_RATE' is not defined

In [42]:
inp_ids = []
lbs = []
idx = 0
for inp in inputsKana.input_ids:
    actual_tokens = list(set(range(100)) - 
                         set(np.where((inp == 101) | (inp == 102) 
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens), 
                                     size=num_of_token_to_mask, 
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
#inp_ids = tf.convert_to_tensor(inp_ids)
inputsKana['input_ids'] = inp_ids

In [46]:
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
history = model.fit([inputsKana.input_ids,inputsKana.attention_mask],inputsKana.labels,verbose=1,batch_size=8,epochs=10)

AttributeError: 'BertModel' object has no attribute 'fit'

1. concatinate kana and kanji text and put a sperator token in between [SEP]
some kana text [SEP] some kanji text

2. randomly mask kanji text
some kana text [SEP] some [MASK] text [MASK]
***[MASK] is identified as 103

3. train model

4. predict a text as so

another kana text [SEP] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]